In [1]:
#!/usr/bin/env python3
"""
Deep Learning Models for Customer Satisfaction Prediction
Implements and compares 6 different deep learning architectures
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support, recall_score, precision_score
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential

from nltk.corpus import wordnet
from tensorflow.keras.layers import (
    Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D,
    Embedding, Dropout, Input, concatenate, Attention, MultiHeadAttention,
    LayerNormalization, Add, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D
)

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    TerminateOnNaN,
    ModelCheckpoint,
    CSVLogger
)

from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import pickle
import warnings
import os
import json
from datetime import datetime
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
# Load preprocessed data
print("Loading preprocessed data...")
data = np.load('data/preprocessed_data.npz')

X_num_train = data['X_num_train']
X_num_val = data['X_num_val']
X_num_test = data['X_num_test']
X_text_train = data['X_text_train']
X_text_val = data['X_text_val']
X_text_test = data['X_text_test']
X_num_train_balanced = data['X_num_train_balanced']
X_text_train_balanced = data['X_text_train_balanced']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']
y_train_balanced = data['y_train_balanced']

# Load TF-IDF based data
X_train_mlp = data['X_train_mlp']
X_val_mlp = data['X_val_mlp']
X_test_mlp = data['X_test_mlp']
y_train_mlp = data['y_train_mlp']
y_val_mlp = data['y_val_mlp']
y_test_mlp = data['y_test_mlp']

# Load metadata
with open('data/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

vocab_size = metadata['vocab_size']
max_len = metadata['max_sequence_length']
num_features = len(metadata['feature_columns'])
num_classes = metadata['num_classes']
class_weights = metadata['class_weights']
class_names = metadata['class_names']

print(f"Vocab size: {vocab_size}")
print(f"Max sequence length: {max_len}")
print(f"Number of features: {num_features}")
print(f"Number of classes: {num_classes}")
print(f"Training with balanced data: {X_text_train_balanced.shape[0]} samples")


Loading preprocessed data...
Vocab size: 27319
Max sequence length: 100
Number of features: 14
Number of classes: 5
Training with balanced data: 19868 samples


In [3]:

class DeepLearningModels:
    def __init__(self, vocab_size, max_len, num_features, num_classes, embedding_dim=128):
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.num_features = num_features
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.models = {}
        self.histories = {}

    def create_mlp_model(self, input_dim):
        """Model 6: Deep MLP with TF-IDF features"""
        model = Sequential([ Input(shape=(input_dim,)),
                  # First hidden layer with batch normalization
                  Dense(1024, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_dim,)),
                  BatchNormalization(),
                  Dropout(0.6),
                  # Second hidden layer
                  Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
                  BatchNormalization(),
                  Dropout(0.5),
                  # Third hidden layer
                  Dense(256, activation='relu'),
                  Dropout(0.4),
                  Dense(128, activation='relu'),
                  # Output layer
                  Dense(self.num_classes, activation='softmax')
        ])
        return model


    def create_lstm_model(self):
        """Model 1: LSTM-based RNN for sequential text processing"""
        # Text input branch
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, self.embedding_dim, mask_zero=True)(text_input)
        x = SpatialDropout1D(0.3)(x)
        x = LSTM(128, return_sequences=True)(x)
        x = LSTM(64)(x)

        # Numerical input branch
        num_input = Input(shape=(self.num_features,))
        y = Dense(64, activation='relu')(num_input)
        y = BatchNormalization()(y)

        # Combine branches
        z = concatenate([x, y])
        z = Dense(128, activation='relu')(z)
        z = Dropout(0.5)(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        optimizer = Adam(learning_rate=0.001, clipvalue=0.5)
        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def create_bilstm_attention_model(self):
        """Model 2: Bidirectional LSTM with attention mechanism"""
        # Text input branch
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, self.embedding_dim, mask_zero=True)(text_input)
        x = Bidirectional(LSTM(128, return_sequences=True))(x)

        # Attention
        attention = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
        x = LayerNormalization()(x + attention)

        # Numerical branch
        num_input = Input(shape=(self.num_features,))
        y = Dense(64, activation='swish')(num_input)
        y = LayerNormalization()(y)

        # Combined
        context = GlobalAveragePooling1D()(x)
        z = concatenate([context, y])
        z = Dense(256, activation='swish')(z)
        z = Dropout(0.4)(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=0.0005),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def create_robust_model(self):
        # Text
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, 128)(text_input)
        x = Bidirectional(LSTM(64))(x)

        # Numerical
        num_input = Input(shape=(self.num_features,))
        y = Dense(64)(num_input)

        # Combined
        z = concatenate([x, y])
        z = Dense(128, activation='relu')(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def create_accurate_bilstm_attention_model(self):
        """High-performance BiLSTM with Attention"""
        # Text input branch
        text_input = Input(shape=(self.max_len,), name='text_input')
        text_embedding = Embedding(
            self.vocab_size,
            self.embedding_dim * 2,  # Increased capacity
            mask_zero=True
        )(text_input)

        # Enhanced Bidirectional LSTM
        bilstm = Bidirectional(
            LSTM(128,  # Doubled units
                 dropout=0.3,
                 recurrent_dropout=0.25,
                 return_sequences=True,
                 kernel_regularizer=l2(1e-4))  # Added regularization
        )(text_embedding)
        bilstm = BatchNormalization()(bilstm)  # Stabilizes training

        # Powerful attention mechanism
        attention = MultiHeadAttention(
            num_heads=8,  # More attention heads
            key_dim=128,  # Matches LSTM units
            dropout=0.2,
            kernel_regularizer=l2(1e-4)
        )(bilstm, bilstm)

        # Residual connection with layer norm
        attention = Add()([bilstm, attention])
        attention = LayerNormalization()(attention)

        # Context extraction
        text_features = GlobalAveragePooling1D()(attention)

        # Enhanced numerical branch
        num_input = Input(shape=(self.num_features,), name='numerical_input')
        num_dense = Dense(64, activation='relu', kernel_regularizer=l2(1e-4))(num_input)
        num_dense = BatchNormalization()(num_dense)

        # Feature fusion
        combined = concatenate([text_features, num_dense])
        combined = Dropout(0.4)(combined)

        # Deep classifier head
        hidden = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(combined)
        hidden = BatchNormalization()(hidden)
        hidden = Dropout(0.4)(hidden)
        hidden = Dense(128, activation='relu')(hidden)
        output = Dense(self.num_classes, activation='softmax')(hidden)

        model = Model(inputs=[text_input, num_input], outputs=output)
        return model

    def create_cnn_model(self):
        """Model 3: CNN for text classification with multiple filter sizes"""
        # Text input branch
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, 128)(text_input)

        # Parallel conv branches with residual connections
        convs = []
        for filter_size in [3, 5, 7]:
            conv = Conv1D(128, filter_size, padding='same', activation='relu')(x)
            conv = MaxPooling1D(2)(conv)
            conv = Conv1D(64, filter_size, padding='same', activation='relu')(conv)
            conv = GlobalMaxPooling1D()(conv)
            convs.append(conv)

        x = concatenate(convs) if len(convs) > 1 else convs[0]

        # Numerical branch
        num_input = Input(shape=(self.num_features,))
        y = Dense(64, activation='relu')(num_input)
        y = BatchNormalization()(y)

        # Combined
        z = concatenate([x, y])
        z = Dense(128, activation='relu')(z)
        z = Dropout(0.5)(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        model.compile(optimizer=RMSprop(learning_rate=0.0005),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def create_transformer_model(self):
        """Model 4: Transformer-based model (simplified BERT-like architecture)"""
        # Text input branch
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, 128)(text_input)

        # Positional encoding
        positions = tf.range(start=0, limit=self.max_len, delta=1)
        positions = Embedding(self.max_len, 128)(positions)
        x = x + positions

        # Transformer blocks
        for _ in range(3):  # Additional layer
            attn = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
            x = LayerNormalization()(x + attn)
            ffn = Dense(512, activation='gelu')(x)
            ffn = Dense(128)(ffn)
            x = LayerNormalization()(x + ffn)

        x = GlobalAveragePooling1D()(x)

        # Numerical branch
        num_input = Input(shape=(self.num_features,))
        y = Dense(64, activation='relu')(num_input)
        y = LayerNormalization()(y)

        # Combined
        z = concatenate([x, y])
        z = Dense(256, activation='gelu')(z)
        z = Dropout(0.4)(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def create_hybrid_cnn_lstm_model(self):
        """Model 5: Hybrid CNN-LSTM model"""
        # Text input branch
        text_input = Input(shape=(self.max_len,))
        x = Embedding(self.vocab_size, 128)(text_input)

        # CNN part
        conv1 = Conv1D(128, 3, padding='same', activation='relu')(x)
        conv1 = MaxPooling1D(2)(conv1)
        conv2 = Conv1D(128, 5, padding='same', activation='relu')(x)
        conv2 = MaxPooling1D(2)(conv2)
        x = concatenate([conv1, conv2])
        x = BatchNormalization()(x)

        # LSTM part
        x = Bidirectional(LSTM(128))(x)

        # Numerical branch
        num_input = Input(shape=(self.num_features,))
        y = Dense(64, activation='relu')(num_input)
        y = BatchNormalization()(y)

        # Combined
        z = concatenate([x, y])
        z = Dense(256, activation='relu')(z)
        z = Dropout(0.5)(z)
        output = Dense(self.num_classes, activation='softmax')(z)

        model = Model(inputs=[text_input, num_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=0.0005),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def create_hybrid_cnn_lstm_model_modified(self):
        """Fixed version with all required imports"""
        # Text input branch
        text_input = Input(shape=(self.max_len,), name='text_input')
        text_embedding = Embedding(self.vocab_size, self.embedding_dim)(text_input)

        # CNN with MaxPooling
        conv1 = Conv1D(128, 3, activation='relu', padding='same')(text_embedding)
        conv1 = MaxPooling1D(2)(conv1)
        conv2 = Conv1D(128, 5, activation='relu', padding='same')(text_embedding)
        conv2 = MaxPooling1D(2)(conv2)

        conv_combined = concatenate([conv1, conv2])
        conv_combined = BatchNormalization()(conv_combined)  # Now properly imported

        # LSTM
        lstm_out = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(conv_combined)

        # Numerical branch
        num_input = Input(shape=(self.num_features,), name='numerical_input')
        num_dense = Dense(32, activation='relu')(num_input)

        # Combine branches
        combined = concatenate([lstm_out, num_dense])
        hidden = Dense(128, activation='relu')(combined)
        output = Dense(self.num_classes, activation='softmax')(hidden)

        model = Model(inputs=[text_input, num_input], outputs=output)
        return model

    def compile_model(self, model, learning_rate=0.001):
        """Compile model with appropriate optimizer and loss function"""
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def create_callbacks(self):
        """Create training callbacks"""
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7
        )

        return [early_stopping, reduce_lr]

    def train_model_mlp(self, model, model_name, X_train, X_val, y_train, y_val,
                    class_weights=None, epochs=100, batch_size=128):
        """Train a model with given data"""
        print(f"\nTraining {model_name}...")

        #callbacks = self.create_callbacks()
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-5),
            TerminateOnNaN()
        ]

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            class_weight=class_weights,
            callbacks=callbacks,
            verbose=1
        )

        self.models[model_name] = model
        self.histories[model_name] = history

        return model, history

    def train_model(self, model, model_name, X_text_train, X_num_train, y_train,
                    X_text_val, X_num_val, y_val, class_weights, epochs=100):
        """Train a model with given data"""
        print(f"\nTraining {model_name}...")

        callbacks = self.create_callbacks()

        history = model.fit(
            [X_text_train, X_num_train], y_train,
            validation_data=([X_text_val, X_num_val], y_val),
            epochs=epochs,
            batch_size=16,
            class_weight=class_weights,
            callbacks=callbacks,
            verbose=1
        )

        self.models[model_name] = model
        self.histories[model_name] = history

        return model, history

    def evaluate_model(self, model, model_name, X_text_test, X_num_test, y_test, class_names):
        """Evaluate model performance"""
        print(f"\nEvaluating {model_name}...")

        # Make predictions - handle MLP vs other models differently
        if 'MLP' in model_name:
            # MLP expects single input array
            y_pred_proba = model.predict(X_text_test)  # X_text_test actually contains all features for MLP
        else:
            # Other models expect separate text and numerical inputs
            y_pred_proba = model.predict([X_text_test, X_num_test])

        # Make predictions
        y_pred = np.argmax(y_pred_proba, axis=1)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1_weighted = f1_score(y_test, y_pred, average='weighted')
        f1_macro = f1_score(y_test, y_pred, average='macro')
        micro_precision = precision_score(y_test, y_pred, average='micro')
        micro_recall = recall_score(y_test, y_pred, average='micro')

        # Multi-class ROC AUC
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            auc_score = 0.0

        # Precision, Recall, F1 per class
        precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average=None)

        # Classification report
        report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        results = {
            'model_name': model_name,
            'accuracy': accuracy,
            'f1_weighted': f1_weighted,
            'f1_macro': f1_macro,
            'auc_score': auc_score,
            'micro_precision': micro_precision,
            'micro_recall': micro_recall,
            'precision': precision,
            'recall': recall,
            'f1_per_class': f1,
            'support': support,
            'classification_report': report,
            'confusion_matrix': cm,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }

        return results

    def plot_training_history(self, model_name):
        """Plot training history"""
        if model_name not in self.histories:
            return

        history = self.histories[model_name]

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        # Plot accuracy
        ax1.plot(history.history['accuracy'], label='Training Accuracy')
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_title(f'{model_name} - Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()
        ax1.grid(True)

        # Plot loss
        ax2.plot(history.history['loss'], label='Training Loss')
        ax2.plot(history.history['val_loss'], label='Validation Loss')
        ax2.set_title(f'{model_name} - Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.savefig(f'charts/{model_name.lower().replace(" ", "_")}_training_history.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

    def plot_confusion_matrix(self, results, class_names):
        """Plot confusion matrix"""
        cm = results['confusion_matrix']
        model_name = results['model_name']

        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names)
        plt.title(f'{model_name} - Confusion Matrix')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.tight_layout()
        plt.savefig(f'charts/{model_name.lower().replace(" ", "_")}_confusion_matrix.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

    def save_all_models(self, all_results):
        """Save all trained models with their evaluation results and supporting files for API use"""
        # Create directories if they don't exist
        os.makedirs('api_models', exist_ok=True)
        os.makedirs('api_models/data', exist_ok=True)

        # Actual features from your Temu reviews dataset
        feature_columns = [
            'ReviewCount', 'UserCountry_encoded',
            'text_length', 'word_count', 'avg_word_length',
            'exclamation_count', 'question_count', 'upper_case_ratio',
            'title_text_length', 'title_word_count', 'title_avg_word_length',
            'title_exclamation_count', 'title_question_count', 'title_upper_case_ratio'
        ]

        # Class names based on ReviewRating (1-5 stars)
        class_names = [
            '1 Star - Very Poor',
            '2 Stars - Poor',
            '3 Stars - Average',
            '4 Stars - Good',
            '5 Stars - Excellent'
        ]

        # Create a package for each model that contains everything needed for serving
        for result in all_results:
            model_name = result['model_name']
            if model_name in self.models:
                # Create a directory for this model
                model_dir = os.path.join('api_models', model_name.lower().replace(' ', '_'))
                os.makedirs(model_dir, exist_ok=True)

                # 1. Save the model in SavedModel format
                model_path = os.path.join(model_dir, 'model.keras')
                self.models[model_name].save(model_path)

                # 2. Save metadata needed for preprocessing
                metadata = {
                    'max_sequence_length': self.max_len,
                    'feature_columns': feature_columns,
                    'class_names': class_names,
                    'input_details': {
                        'text_input': {
                            'shape': [None, self.max_len],
                            'dtype': 'int32',
                            'description': 'Tokenized review text from ReviewText column'
                        },
                        'numerical_input': {
                            'shape': [None, len(feature_columns)],
                            'dtype': 'float32',
                            'description': f'Numerical features in order: {", ".join(feature_columns)}'
                        }
                    },
                    'output_details': {
                        'description': 'Probability scores for each rating level (1-5 stars)',
                        'class_order': class_names
                    },
                    'data_source': 'temu_reviews_cleaned.csv',
                    'text_columns_used': ['ReviewText', 'ReviewTitle'],  # Which text columns were used
                    'model_format': 'keras'  # Indicate the saved format
                }

                with open(os.path.join(model_dir, 'metadata.json'), 'w') as f:
                    json.dump(metadata, f, indent=2)

                print(f"✅ Saved API-ready {model_name} package to {model_dir}")

                # Update the result with the path
                result['api_model_path'] = model_dir
            else:
                print(f"⚠️ Model {model_name} not found in trained models")

        # Save tokenizer if exists
        if hasattr(self, 'tokenizer'):
            tokenizer_path = os.path.join('api_models', 'tokenizer.pkl')
            with open(tokenizer_path, 'wb') as f:
                pickle.dump(self.tokenizer, f)
            print(f"✅ Saved tokenizer to {tokenizer_path}")

        # Save complete results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_path = os.path.join('api_models', 'data', f'model_results_{timestamp}.pkl')
        with open(results_path, 'wb') as f:
            pickle.dump(all_results, f)

        print(f"\nAll models saved in API-ready format.")
        print(f"You can now deploy any model by copying its directory to your API server.")
        return results_path





In [ ]:
# Define models to train
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes, embedding_dim=128)

#'Deep MLP with TF-IDF'
# Define models to train
model_configs = [
    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1]))
]

# Train and evaluate all models
all_results = []

for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()


    # Special handling for MLP model (uses different data)
    if 'MLP' in model_name:
        # Handle class imbalance for MLP data
        smote = SMOTE(random_state=42)
        X_train_mlp_balanced, y_train_mlp_balanced = smote.fit_resample(X_train_mlp, y_train_mlp)

        # Train MLP model
        model, history = model_builder.train_model_mlp(
            model, model_name,
            X_train_mlp_balanced, X_val_mlp, y_train_mlp_balanced, y_val_mlp,
            class_weights, epochs=50, batch_size=64
        )

        # Evaluate MLP model
        results = model_builder.evaluate_model(
            model, model_name, X_test_mlp, None, y_test_mlp, class_names
        )
    else:
        # Train sequence-based models
        model, history = model_builder.train_model(
            model, model_name,
            X_text_train_balanced, X_num_train_balanced, y_train_balanced,
            X_text_val, X_num_val, y_val,
            class_weights, epochs=50
        )
        # Evaluate model
        results = model_builder.evaluate_model(
            model, model_name, X_text_test, X_num_test, y_test, class_names
        )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    print(results.keys())  # Check available keys
    print(f"Micro-averaged precision: {results['micro_precision']:.4f}")
    print(f"Micro-average recall: {results['micro_recall']:.4f}")
    print("Precision:", ", ".join([f"{x:.4f}" for x in results['precision']]))
    print("Recall:", ", ".join([f"{x:.4f}" for x in results['recall']]))
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    



Building and training Deep MLP with TF-IDF

Deep MLP with TF-IDF Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │      10,255,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,951,173 (41.78 MB)

 Trainable params: 10,948,101 (41.76 MB)

 Non-trainable params: 3,072 (12.00 KB)


Training Deep MLP with TF-IDF...
Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 116s 258ms/step - accuracy: 0.3451 - loss: 4.2985 - val_accuracy: 0.5074 - val_loss: 2.9889 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 79s 204ms/step - accuracy: 0.7052 - loss: 2.3900 - val_accuracy: 0.3897 - val_loss: 3.4735 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 116s 297ms/step - accuracy: 0.8014 - loss: 2.0905 - val_accuracy: 0.5022 - val_loss: 3.4328 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 173s 373ms/step - accuracy: 0.8329 - loss: 2.0195 - val_accuracy: 0.5662 - val_loss: 3.0996 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 203s 373ms/step - accuracy: 0.8569 - loss: 1.9327 - val_accuracy: 0.5757 - val_loss: 2.9623 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 120s 133ms/step - accuracy: 0.8616 - loss: 1.8285 - val_accuracy: 0.5750 - val_loss: 2.8377 - learning_rate: 0.0010
Epoch 7/50
388/388 ━━━━━━━━━━━━

In [5]:
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes)

#'BiLSTM with Attention', model_builder.create_bilstm_attention_model)
# Define models to train
model_configs = [
    ('LSTM Model', model_builder.create_lstm_model)
]

# Train and evaluate all models
for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()
    
    # Train model
    model, history = model_builder.train_model(
        model, model_name,
        X_text_train_balanced, X_num_train_balanced, y_train_balanced,
        X_text_val, X_num_val, y_val,
        class_weights, epochs=50
    )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Evaluate model
    results = model_builder.evaluate_model(
        model, model_name, X_text_test, X_num_test, y_test, class_names
    )
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    



1242/1242 ━━━━━━━━━━━━━━━━━━━━ 264s 180ms/step - accuracy: 0.2783 - loss: 2.2227 - val_accuracy: 0.2346 - val_loss: 2.0503 - learning_rate: 0.0010
Epoch 2/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 277s 191ms/step - accuracy: 0.3971 - loss: 1.8358 - val_accuracy: 0.5243 - val_loss: 1.5521 - learning_rate: 0.0010
Epoch 3/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 269s 197ms/step - accuracy: 0.5071 - loss: 1.6593 - val_accuracy: 0.4301 - val_loss: 1.7800 - learning_rate: 0.0010
Epoch 4/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 264s 198ms/step - accuracy: 0.5627 - loss: 1.5420 - val_accuracy: 0.5162 - val_loss: 1.7360 - learning_rate: 0.0010
Epoch 5/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 251s 189ms/step - accuracy: 0.6088 - loss: 1.4218 - val_accuracy: 0.5184 - val_loss: 1.7840 - learning_rate: 0.0010
Epoch 6/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 214s 172ms/step - accuracy: 0.6302 - loss: 1.3170 - val_accuracy: 0.5243 - val_loss: 1.7457 - learning_rate: 0.0010
Epoch 7/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 259s 169ms/step - acc

In [6]:
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes)

#'CNN Model', model_builder.create_cnn_model)
# Define models to train
model_configs = [
    ('BiLSTM with Attention', model_builder.create_robust_model)
]

# Train and evaluate all models

for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()
    
    # Train model
    model, history = model_builder.train_model(
        model, model_name,
        X_text_train_balanced, X_num_train_balanced, y_train_balanced,
        X_text_val, X_num_val, y_val,
        class_weights, epochs=50
    )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Evaluate model
    results = model_builder.evaluate_model(
        model, model_name, X_text_test, X_num_test, y_test, class_names
    )
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    




Building and training BiLSTM with Attention

BiLSTM with Attention Architecture:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 100, 128)          │       3,496,832 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_4 (InputLayer)    │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 128)               │          98,816 │ embedding_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 64)                │             960 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 192)               │               0 │ bidirectional[0][0],       │
│                               │                           │                 │ dense_8[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 128)               │          24,704 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 5)                 │             645 │ dense_9[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 3,621,957 (13.82 MB)

 Trainable params: 3,621,957 (13.82 MB)

 Non-trainable params: 0 (0.00 B)


Training BiLSTM with Attention...
Epoch 1/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 110s 72ms/step - accuracy: 0.3171 - loss: 2.0375 - val_accuracy: 0.4250 - val_loss: 1.8026 - learning_rate: 0.0010
Epoch 2/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 126s 101ms/step - accuracy: 0.6201 - loss: 1.3534 - val_accuracy: 0.4206 - val_loss: 2.0618 - learning_rate: 0.0010
Epoch 4/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 138s 98ms/step - accuracy: 0.6847 - loss: 1.0949 - val_accuracy: 0.5029 - val_loss: 1.8362 - learning_rate: 0.0010
Epoch 5/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 121s 97ms/step - accuracy: 0.7383 - loss: 0.8558 - val_accuracy: 0.5463 - val_loss: 1.8579 - learning_rate: 0.0010
Epoch 6/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 150s 104ms/step - accuracy: 0.7761 - loss: 0.6987 - val_accuracy: 0.6074 - val_loss: 1.8134 - learning_rate: 0.0010
Epoch 7/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 131s 105ms/step - accuracy: 0.8337 - loss: 0.4798 - val_accuracy: 0.5831 - val_loss: 2.3132 - learning_rate: 0.0010
Epoch 9/50
1242/1242

In [7]:
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes)

#'Transformer Model', model_builder.create_transformer_model)
# Define models to train
model_configs = [
    ('CNN Model', model_builder.create_cnn_model)
]

# Train and evaluate all models

for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()
    
    # Train model
    model, history = model_builder.train_model(
        model, model_name,
        X_text_train_balanced, X_num_train_balanced, y_train_balanced,
        X_text_val, X_num_val, y_val,
        class_weights, epochs=50
    )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Evaluate model
    results = model_builder.evaluate_model(
        model, model_name, X_text_test, X_num_test, y_test, class_names
    )
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    




Building and training CNN Model

CNN Model Architecture:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)    │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_2 (Embedding)       │ (None, 100, 128)          │       3,496,832 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 100, 128)          │          49,280 │ embedding_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 100, 128)          │          82,048 │ embedding_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_4 (Conv1D)             │ (None, 100, 128)          │         114,816 │ embedding_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 50, 128)           │               0 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_1               │ (None, 50, 128)           │               0 │ conv1d_2[0][0]             │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_2               │ (None, 50, 128)           │               0 │ conv1d_4[0][0]             │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 50, 64)            │          24,640 │ max_pooling1d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, 50, 64)            │          41,024 │ max_pooling1d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_5 (Conv1D)             │ (None, 50, 64)            │          57,408 │ max_pooling1d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_6 (InputLayer)    │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 64)                │               0 │ conv1d_1[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_1        │ (None, 64)                │               0 │ conv1d_3[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_2        │ (None, 64)                │               0 │ conv1d_5[0][0]             │
│ (GlobalMaxPooling1D)          │                           │               

 Total params: 3,900,805 (14.88 MB)

 Trainable params: 3,900,677 (14.88 MB)

 Non-trainable params: 128 (512.00 B)


Training CNN Model...
Epoch 1/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 117s 91ms/step - accuracy: 0.2800 - loss: 2.2053 - val_accuracy: 0.1029 - val_loss: 2.4103 - learning_rate: 0.0010
Epoch 2/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 109s 88ms/step - accuracy: 0.5401 - loss: 1.5624 - val_accuracy: 0.5596 - val_loss: 1.5740 - learning_rate: 0.0010
Epoch 4/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 109s 88ms/step - accuracy: 0.7185 - loss: 0.9241 - val_accuracy: 0.5507 - val_loss: 1.8061 - learning_rate: 0.0010
Epoch 6/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 99s 80ms/step - accuracy: 0.7706 - loss: 0.7001 - val_accuracy: 0.5537 - val_loss: 1.8578 - learning_rate: 0.0010
Epoch 7/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 106s 85ms/step - accuracy: 0.7978 - loss: 0.6015 - val_accuracy: 0.5912 - val_loss: 1.7768 - learning_rate: 0.0010
Epoch 8/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 93s 75ms/step - accuracy: 0.8203 - loss: 0.5176 - val_accuracy: 0.6118 - val_loss: 1.9188 - learning_rate: 0.0010
Epoch 9/50
1242/1242 ━━━━━━━━━━━━━━━━

In [8]:
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes)

#'Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
# Define models to train
model_configs = [
    ('Transformer Model', model_builder.create_transformer_model)
]

# Train and evaluate all models

for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()
    
    # Train model
    model, history = model_builder.train_model(
        model, model_name,
        X_text_train_balanced, X_num_train_balanced, y_train_balanced,
        X_text_val, X_num_val, y_val,
        class_weights, epochs=50
    )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Evaluate model
    results = model_builder.evaluate_model(
        model, model_name, X_text_test, X_num_test, y_test, class_names
    )
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    




Building and training Transformer Model

Transformer Model Architecture:


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)    │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, 100, 128)          │       3,496,832 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 100, 128)          │               0 │ embedding_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 100, 128)          │          66,048 │ add[0][0], add[0][0]       │
│ (MultiHeadAttention)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 100, 128)          │               0 │ add[0][0],                 │
│                               │                           │                 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 100, 128)          │             256 │ add_1[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_14 (Dense)              │ (None, 100, 512)          │          66,048 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_15 (Dense)              │ (None, 100, 128)          │          65,664 │ dense_14[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_2 (Add)                   │ (None, 100, 128)          │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ dense_15[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 100, 128)          │             256 │ add_2[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_1        │ (None, 100, 128)          │          66,048 │ layer_normalization_1[0][… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_3 (Add)                   │ (None, 100, 128)          │               0 │ layer_normalization_1[0][… │
│                               │                           │                 │ multi_head_attention_1[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_2         │ (None, 100, 128)          │             256 │ add_3[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 4,143,429 (15.81 MB)

 Trainable params: 4,143,429 (15.81 MB)

 Non-trainable params: 0 (0.00 B)


Training Transformer Model...
Epoch 1/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 431s 291ms/step - accuracy: 0.2895 - loss: 2.1586 - val_accuracy: 0.0941 - val_loss: 2.4799 - learning_rate: 0.0010
Epoch 2/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 354s 285ms/step - accuracy: 0.3401 - loss: 1.9044 - val_accuracy: 0.1250 - val_loss: 2.4028 - learning_rate: 0.0010
Epoch 3/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 356s 287ms/step - accuracy: 0.3662 - loss: 1.8333 - val_accuracy: 0.1140 - val_loss: 2.5543 - learning_rate: 0.0010
Epoch 4/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 368s 275ms/step - accuracy: 0.3708 - loss: 1.8128 - val_accuracy: 0.1199 - val_loss: 2.4856 - learning_rate: 0.0010
Epoch 5/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 397s 287ms/step - accuracy: 0.4011 - loss: 1.7055 - val_accuracy: 0.1382 - val_loss: 2.4695 - learning_rate: 0.0010
Epoch 6/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 399s 300ms/step - accuracy: 0.4147 - loss: 1.6487 - val_accuracy: 0.1382 - val_loss: 2.4450 - learning_rate: 0.0010
Epoch 7/50
1242/1242 

In [9]:
#model_configs = [
#    ('Deep MLP with TF-IDF', lambda: model_builder.create_mlp_model(X_train_mlp.shape[1])),
#    ('LSTM Model', model_builder.create_lstm_model),
#    ('BiLSTM with Attention', model_builder.create_robust_model),
#    ('CNN Model', model_builder.create_cnn_model),
#    ('Transformer Model', model_builder.create_transformer_model),
#    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
#]


# Initialize model builder
model_builder = DeepLearningModels(vocab_size, max_len, num_features, num_classes)

#'Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
# Define models to train
model_configs = [
    ('Hybrid CNN-LSTM', model_builder.create_hybrid_cnn_lstm_model)
]

# Train and evaluate all models

for model_name, model_func in model_configs:
    print(f"\n{'='*50}")
    print(f"Building and training {model_name}")
    print('='*50)
    
    # Create and compile model
    model = model_func()
    model = model_builder.compile_model(model)
    
    print(f"\n{model_name} Architecture:")
    model.summary()
    
    # Train model
    model, history = model_builder.train_model(
        model, model_name,
        X_text_train_balanced, X_num_train_balanced, y_train_balanced,
        X_text_val, X_num_val, y_val,
        class_weights, epochs=50
    )
    
    # Plot training history
    model_builder.plot_training_history(model_name)
    
    # Evaluate model
    results = model_builder.evaluate_model(
        model, model_name, X_text_test, X_num_test, y_test, class_names
    )
    
    # Plot confusion matrix
    model_builder.plot_confusion_matrix(results, class_names)
    
    all_results.append(results)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"F1-Score (Weighted): {results['f1_weighted']:.4f}")
    print(f"F1-Score (Macro): {results['f1_macro']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Save results
    with open('data/model_results.pkl', 'wb') as f:
        pickle.dump(all_results, f)
    




Building and training Hybrid CNN-LSTM

Hybrid CNN-LSTM Architecture:


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)    │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_5 (Embedding)       │ (None, 100, 128)          │       3,496,832 │ input_layer_9[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_6 (Conv1D)             │ (None, 100, 128)          │          49,280 │ embedding_5[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_7 (Conv1D)             │ (None, 100, 128)          │          82,048 │ embedding_5[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_3               │ (None, 50, 128)           │               0 │ conv1d_6[0][0]             │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_4               │ (None, 50, 128)           │               0 │ conv1d_7[0][0]             │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_5 (Concatenate)   │ (None, 50, 256)           │               0 │ max_pooling1d_3[0][0],     │
│                               │                           │                 │ max_pooling1d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_10 (InputLayer)   │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 50, 256)           │           1,024 │ concatenate_5[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_23 (Dense)              │ (None, 64)                │             960 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_1               │ (None, 256)               │         394,240 │ batch_normalization_4[0][… │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_5         │ (None, 64)                │             256 │ dense_23[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_6 (Concatenate)   │ (None, 320)               │               0 │ bidirectional_1[0][0],     │
│                               │                           │                 │ batch_normalization_5[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_24 (Dense)              │ (None, 256)               │          82,17

 Total params: 4,108,101 (15.67 MB)

 Trainable params: 4,107,461 (15.67 MB)

 Non-trainable params: 640 (2.50 KB)


Training Hybrid CNN-LSTM...
Epoch 1/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 105s 80ms/step - accuracy: 0.3014 - loss: 2.1519 - val_accuracy: 0.3985 - val_loss: 1.8427 - learning_rate: 0.0010
Epoch 2/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 100s 81ms/step - accuracy: 0.4753 - loss: 1.7171 - val_accuracy: 0.4824 - val_loss: 1.6117 - learning_rate: 0.0010
Epoch 3/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 100s 81ms/step - accuracy: 0.5906 - loss: 1.4349 - val_accuracy: 0.5132 - val_loss: 1.7146 - learning_rate: 0.0010
Epoch 4/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 101s 81ms/step - accuracy: 0.6751 - loss: 1.1104 - val_accuracy: 0.5184 - val_loss: 1.7820 - learning_rate: 0.0010
Epoch 5/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 101s 81ms/step - accuracy: 0.7401 - loss: 0.8324 - val_accuracy: 0.5184 - val_loss: 2.0323 - learning_rate: 0.0010
Epoch 6/50
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 101s 82ms/step - accuracy: 0.7833 - loss: 0.6461 - val_accuracy: 0.6103 - val_loss: 1.8043 - learning_rate: 0.0010
Epoch 7/50
1242/1242 ━━━━━━━━

In [10]:
# Create comparison summary
summary_data = []
for result in all_results:
    summary_data.append({
        'Model': result['model_name'],
        'Accuracy': result['accuracy'],
        'F1-Weighted': result['f1_weighted'],
        'F1-Macro': result['f1_macro'],
        'AUC Score': result['auc_score']
    })

summary_df = pd.DataFrame(summary_data)
print(all_results)
#summary_df = summary_df.sort_values('F1-Weighted', ascending=False)


[{'model_name': 'Deep MLP with TF-IDF', 'accuracy': 0.6609047443913203, 'f1_weighted': 0.6850760177709129, 'f1_macro': 0.43772467048663505, 'auc_score': np.float64(0.8637367863754088), 'micro_precision': 0.6609047443913203, 'micro_recall': 0.6609047443913203, 'precision': array([0.87340764, 0.18503937, 0.11711712, 0.22779923, 0.78021978]), 'recall': array([0.77471751, 0.27647059, 0.20155039, 0.26818182, 0.7244898 ]), 'f1_per_class': array([0.82110778, 0.22169811, 0.14814815, 0.24634656, 0.75132275]), 'support': array([1416,  170,  129,  220,  784]), 'classification_report': {'Rating 1': {'precision': 0.8734076433121019, 'recall': 0.7747175141242938, 'f1-score': 0.8211077844311377, 'support': 1416.0}, 'Rating 2': {'precision': 0.18503937007874016, 'recall': 0.27647058823529413, 'f1-score': 0.22169811320754718, 'support': 170.0}, 'Rating 3': {'precision': 0.11711711711711711, 'recall': 0.20155038759689922, 'f1-score': 0.14814814814814814, 'support': 129.0}, 'Rating 4': {'precision': 0.22

In [11]:
# Create comparison summary
summary_data = []
for result in all_results:
    summary_data.append({
        'Model': result['model_name'],
        'Accuracy': result['accuracy'],
        'F1-Weighted': result['f1_weighted'],
        'F1-Macro': result['f1_macro'],
        'AUC Score': result['auc_score']
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('F1-Weighted', ascending=False)

print(f"\n{'='*80}")
print("MODEL PERFORMANCE COMPARISON")
print('='*80)
print(summary_df.to_string(index=False, float_format='%.4f'))

# Save summary
summary_df.to_csv('data/model_comparison_summary.csv', index=False)

# Plot comparison
plt.figure(figsize=(14, 8))

metrics = ['Accuracy', 'F1-Weighted', 'F1-Macro', 'AUC Score']
x = np.arange(len(summary_df))
width = 0.2

for i, metric in enumerate(metrics):
    plt.bar(x + i*width, summary_df[metric], width, label=metric, alpha=0.8)

plt.xlabel('Models')
plt.ylabel('Score')
plt.title('Deep Learning Models Performance Comparison')
plt.xticks(x + width*1.5, summary_df['Model'], rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('charts/model_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"\nBest performing model: {summary_df.iloc[0]['Model']}")
print(f"Best F1-Weighted Score: {summary_df.iloc[0]['F1-Weighted']:.4f}")

# After training all models, save everything
results_path = model_builder.save_all_models(all_results)
print(f"All models and results saved. Results path: {results_path}")
    
    




MODEL PERFORMANCE COMPARISON
                Model  Accuracy  F1-Weighted  F1-Macro  AUC Score
 Deep MLP with TF-IDF    0.6609       0.6851    0.4377     0.8637
            CNN Model    0.5337       0.5904    0.3456     0.8626
           LSTM Model    0.5094       0.5776    0.3517     0.8754
      Hybrid CNN-LSTM    0.4678       0.5441    0.3173     0.8453
BiLSTM with Attention    0.4546       0.5240    0.3190     0.8515
    Transformer Model    0.2041       0.2376    0.1615     0.5741

Best performing model: Deep MLP with TF-IDF
Best F1-Weighted Score: 0.6851
⚠️ Model Deep MLP with TF-IDF not found in trained models
⚠️ Model LSTM Model not found in trained models
⚠️ Model BiLSTM with Attention not found in trained models
⚠️ Model CNN Model not found in trained models
⚠️ Model Transformer Model not found in trained models
✅ Saved API-ready Hybrid CNN-LSTM package to api_models\hybrid_cnn-lstm

All models saved in API-ready format.
You can now deploy any model by copying its directory 